# Evaluación de Modelos de Clasificación de Género

Este cuaderno carga los conjuntos de datos de entrenamiento y prueba, y a continuación evalúa los siguientes modelos:
1. **Modelos Básicos**: Random Forest, Naive Bayes, Logistic Regression, Linear SVM.
2. **Modelo Transformer**: DeBERTa-v3-large.

Si los modelos no existen previamente, se entrenarán automáticamente.

In [1]:
import pandas as pd
import os
import torch
import shutil
from typing import List
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from src.genre_classification.F_Basic_Models import Basic_Models
from src.genre_classification.F_Dataset_Downloader import Dataset_Downloader
from src.genre_classification.F_Pretrained_models import Pretrained
from src.genre_classification.F_Compute_Metrics import Compute_Metrics

c:\Users\alber\Desktop\CUARTO CURSO\PRIMER CUATRIMESTRE\Procesamiento del lenguaje natural II\Practica 1 NLP II\NLP_II_Practica1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Carga de Datos
Cargamos los datos de entrenamiento y prueba. El set de entrenamiento sólo se usa si es necesario entrenar algún modelo.

In [2]:
dataset_downloader = Dataset_Downloader()
train_path, test_path = dataset_downloader(overwrite=False)

print(f"Cargando Train: {train_path}")
print(f"Cargando Test: {test_path}")

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

x_train, y_train = train_data.drop(columns=["genre"]), train_data["genre"]
x_test, y_test = test_data.drop(columns=["genre"]), test_data["genre"]

unique_labels = sorted(list(set(y_train)))
print(f"Etiquetas: {unique_labels}")

Cargando Train: C:\Users\alber\Desktop\CUARTO CURSO\PRIMER CUATRIMESTRE\Procesamiento del lenguaje natural II\Practica 1 NLP II\NLP_II_Practica1\datasets\dataset_train.csv
Cargando Test: C:\Users\alber\Desktop\CUARTO CURSO\PRIMER CUATRIMESTRE\Procesamiento del lenguaje natural II\Practica 1 NLP II\NLP_II_Practica1\datasets\dataset_test.csv
Etiquetas: ['action_adventure', 'comedy_family', 'documentary_factual', 'drama_romance', 'scifi_horror_fantasy', 'suspense_crime']


## 2. Modelos Básicos
Iteramos sobre cada uno de los tipos de modelos básicos. Si el modelo ya está guardado, lo cargamos. Si no, lo entrenamos y guardamos.

In [3]:
basic_models_names = ['Naive_Bayes', 'LogReg', 'Linear_SVM', 'Random_Forest']
results = {}

for model_name in basic_models_names:
    print(f"\n{'='*20} {model_name} {'='*20}")
    model = Basic_Models(model_type=model_name)
    
    model_file = f"./Models/Modelos_Basicos/{model_name}.joblib"
    
    if os.path.exists(model_file):
        print(f"Cargando modelo guardado desde {model_file}...")
        model.load_model(name=f"{model_name}.joblib")
    else:
        print(f"Modelo no encontrado. Entrenando {model_name}...")
        model.fit(x_train, y_train)
        model.save_model(name=model_name)
    

    print(f"Realizando predicciones con {model_name}...")
    y_hat = model.predict(x_test)
    results[model_name] = y_hat
    
    print(f"Evaluación de {model_name}:")
    metrics = model.evaluate(y_true=y_test, y_hat=y_hat, labels=set(y_test), evaluate_type="sk_learn_metrics")
    # print(metrics) # Descomentar para ver reporte detallado aquí


==================== Naive_Bayes ====================
NLTK configurado exitosamente.
Cargando modelo guardado desde ./Models/Modelos_Basicos/Naive_Bayes.joblib...
Modelo cargado correctamente.
Realizando predicciones con Naive_Bayes...
Iniciando predicción...
-> Predicción terminada.
Evaluación de Naive_Bayes:

==================== LogReg ====================
NLTK configurado exitosamente.
Cargando modelo guardado desde ./Models/Modelos_Basicos/LogReg.joblib...
Modelo cargado correctamente.
Realizando predicciones con LogReg...
Iniciando predicción...
-> Predicción terminada.
Evaluación de LogReg:

==================== Linear_SVM ====================
NLTK configurado exitosamente.
Cargando modelo guardado desde ./Models/Modelos_Basicos/Linear_SVM.joblib...
Modelo cargado correctamente.
Realizando predicciones con Linear_SVM...
Iniciando predicción...
-> Predicción terminada.
Evaluación de Linear_SVM:

==================== Random_Forest ====================
NLTK configurado exitosament

## 3. Modelo Transformer (DeBERTa)
Verificamos si existe el modelo pre-entrenado. Si no, realizamos el fine-tuning.

In [ ]:
transformer_path = "./Models/Modelos Transformer/deberta-v3-large"
model_name = "deberta-v3-large"

train_texts = train_data["text"].tolist()
test_texts = test_data["text"].tolist()

# Instanciar la clase wrapper
transformer_wrapper = Pretrained(model_type=model_name, labels=unique_labels)

if os.path.exists(transformer_path) and len(os.listdir(transformer_path)) > 0:
    print(f"\nCargando Transformer guardado desde {transformer_path}...")
    # Carga manual de pesos guardados en la estructura esperada por el wrapper
    transformer_wrapper.model = AutoModelForSequenceClassification.from_pretrained(transformer_path)
    transformer_wrapper.tokenizer = AutoTokenizer.from_pretrained(transformer_path)
    transformer_wrapper.model.to(transformer_wrapper.device)
else:
    print("\nModelo Transformer no encontrado. Iniciando Fine-Tuning...")
    # Preparar labels como índices
    train_labels_ids = [transformer_wrapper.label2id[l] for l in y_train]
    
    # Entrenar (Se recomienda GPU, puede tardar)
    transformer_wrapper.fit(
        train_texts=train_texts,
        train_labels=train_labels_ids,
        batch_size=4,
        epochs=3,
        learning_rate=1e-5
    )
    
    # Guardar modelo
    transformer_wrapper.save_model(path=transformer_path)

print("Realizando inferencia con Transformer...")
# Transform devuelve un array de IDs, necesitamos convertir a labels si queremos comparar strings
predictions_ids = transformer_wrapper.transform(test_texts, batch_size=8)
predictions_labels = [transformer_wrapper.id2label[pid] for pid in predictions_ids]

results['Transformer'] = predictions_labels

## 4. Comparación Final
Mostramos un resumen de las métricas de todos los modelos.

In [ ]:
final_metrics = []

for model_name, preds in results.items():
    cm = Compute_Metrics(y_pred=preds, y_true=y_test, labels=unique_labels)
    metrics_dict = cm.compute_all()
    
    final_metrics.append({
        "Model": model_name,
        "Accuracy": metrics_dict["accuracy"],
        "Macro F1": metrics_dict["macro_f1"]
    })

df_metrics = pd.DataFrame(final_metrics)
df_metrics = df_metrics.sort_values(by="Macro F1", ascending=False)

print("\n--- Tabla Comparativa de Resultados ---")
display(df_metrics)

# Opcional: Mostrar matriz de confusión del mejor modelo
best_model = df_metrics.iloc[0]['Model']
print(f"\nMatriz de Confusión del mejor modelo ({best_model}):")
cm_best = Compute_Metrics(y_pred=results[best_model], y_true=y_test, labels=unique_labels)
display(cm_best.confusion_matrix())